# 🛒 Sudamall E-Commerce: Orders, Payments & Carts Usage Guide

Welcome to the comprehensive guide for using the **Orders**, **Payments**, and **Carts** modules in the Sudamall E-Commerce platform. This notebook demonstrates real-world usage patterns, API interactions, and best practices.

## 📋 What You'll Learn

- **Cart Management**: Add/update/remove items, handle size variations
- **Order Processing**: Complete order lifecycle from checkout to completion
- **Payment Integration**: Multi-gateway payment processing with timers
- **Stock Management**: Atomic operations and concurrent handling
- **Real-time Synchronization**: Cart sync and product change detection
- **Error Handling**: Proper validation and exception management

## 🏗️ Architecture Overview

```
┌─────────────┐    ┌─────────────┐    ┌─────────────┐
│    Cart     │───▶│   Orders    │───▶│  Payments   │
│  Management │    │ Processing  │    │ Integration │
└─────────────┘    └─────────────┘    └─────────────┘
       │                   │                   │
       ▼                   ▼                   ▼
┌─────────────┐    ┌─────────────┐    ┌─────────────┐
│    Stock    │    │  Product    │    │   Timer     │
│ Management  │    │  History    │    │   System    │
└─────────────┘    └─────────────┘    └─────────────┘
```

---

**Let's dive into the practical examples!** 🚀

# 1. 📦 Import Required Libraries

First, let's import all the necessary Django models, serializers, and libraries we'll need for working with the e-commerce system.

In [ ]:
# Core Django imports
import django
import os
import sys
from decimal import Decimal
from datetime import datetime, timedelta
from django.utils import timezone

# Add the project path to Python path
sys.path.append('/home/mahmoudadam/Desktop/E-Commerc-Sudan/back-end/api')

# Django models and utilities
from django.db import transaction
from django.core.exceptions import ValidationError
from django.contrib.auth import get_user_model

# E-commerce app models
from carts.models import Cart, CartItem
from orders.models import Order
from payments.models import Payment, PaymentGateway, PaymentAttempt, Refund
from products.models import Product, Size, ProductHistory
from stores.models import Store
from accounts.models import BusinessOwner

# Services and managers
from carts.services import CartService
from orders.services import OrderService
from payments.services import PaymentService
from products.services.stock_service import StockService

# REST Framework for API interactions
from rest_framework.test import APIClient
from rest_framework import status

# Utility imports
import json
import uuid
from typing import Dict, List, Optional

print("✅ All libraries imported successfully!")
print(f"Django version: {django.get_version()}")
print(f"Current timezone: {timezone.get_current_timezone()}")

# 2. ⚙️ Setup Django Environment

Configure Django settings and database connections to interact with the e-commerce models.

In [ ]:
# Setup Django environment
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'api.settings')
django.setup()

# Get User model
User = get_user_model()

# Create API client for testing
api_client = APIClient()

print("🚀 Django environment configured successfully!")

# Database connection test
try:
    user_count = User.objects.count()
    print(f"📊 Database connected. Total users: {user_count}")
except Exception as e:
    print(f"❌ Database connection error: {e}")

# Helper function to create test data
def create_test_data():
    """Create sample data for demonstration purposes"""
    
    # Create test user
    user, created = User.objects.get_or_create(
        email='buyer@example.com',
        defaults={
            'first_name': 'John',
            'last_name': 'Buyer', 
            'phone_number': '+1234567890',
            'account_type': 'buyer'
        }
    )
    if created:
        user.set_password('testpass123')
        user.save()
    
    # Create business owner and store
    business_user, created = User.objects.get_or_create(
        email='seller@example.com',
        defaults={
            'first_name': 'Jane',
            'last_name': 'Seller',
            'phone_number': '+1234567891', 
            'account_type': 'business'
        }
    )
    if created:
        business_user.set_password('testpass123')
        business_user.save()
    
    # Create store
    store, created = Store.objects.get_or_create(
        name='Demo Store',
        defaults={'location': 'Khartoum, Sudan'}
    )
    
    # Create business owner
    business_owner, created = BusinessOwner.objects.get_or_create(
        user=business_user,
        defaults={'store': store}
    )
    
    # Create test products
    from django.core.files.uploadedfile import SimpleUploadedFile
    test_image = SimpleUploadedFile(
        name='test.jpg',
        content=b'\x47\x49\x46\x38\x39\x61\x01\x00\x01\x00\x80\x00\x00\x00\x00\x00\xFF\xFF\xFF',
        content_type='image/jpeg'
    )
    
    # Product without sizes
    product1, created = Product.objects.get_or_create(
        product_name='Test Laptop',
        defaults={
            'product_description': 'High-performance laptop for coding',
            'price': Decimal('1299.99'),
            'color': 'Silver',
            'brand': 'TechBrand',
            'available_quantity': 10,
            'reserved_quantity': 0,
            'has_sizes': False,
            'owner_id': business_user,
            'store': store,
            'category': 'Electronics',
            'picture': test_image
        }
    )
    
    # Product with sizes
    product2, created = Product.objects.get_or_create(
        product_name='Premium T-Shirt',
        defaults={
            'product_description': 'Comfortable cotton t-shirt',
            'price': Decimal('29.99'),
            'color': 'Blue',
            'brand': 'FashionCo',
            'has_sizes': True,
            'owner_id': business_user,
            'store': store,
            'category': 'Clothing',
            'picture': test_image
        }
    )
    
    if created:
        # Add sizes for the t-shirt
        Size.objects.get_or_create(
            product=product2,
            size='S',
            defaults={'available_quantity': 5, 'reserved_quantity': 0}
        )
        Size.objects.get_or_create(
            product=product2,
            size='M', 
            defaults={'available_quantity': 8, 'reserved_quantity': 0}
        )
        Size.objects.get_or_create(
            product=product2,
            size='L',
            defaults={'available_quantity': 3, 'reserved_quantity': 0}
        )
    
    # Create payment gateway
    gateway, created = PaymentGateway.objects.get_or_create(
        name='test_gateway',
        defaults={
            'gateway_type': 'test_gateway',
            'is_test_mode': True,
            'is_active': True,
            'fixed_fee': Decimal('2.00'),
            'percentage_fee': Decimal('2.5')
        }
    )
    
    return {
        'user': user,
        'business_user': business_user,
        'store': store,
        'product1': product1,
        'product2': product2,
        'gateway': gateway
    }

# Create test data
test_data = create_test_data()
print("✅ Test data created successfully!")
print(f"👤 Buyer: {test_data['user'].email}")
print(f"🏪 Seller: {test_data['business_user'].email}")
print(f"📦 Products: {test_data['product1'].product_name}, {test_data['product2'].product_name}")

# 3. 🛒 Cart Management Operations

Demonstrate adding items to cart, updating quantities, removing items, and managing size-based product variations.

## Key Features:
- **Real-time Stock Validation**: Check availability before adding
- **Size Variations**: Handle products with multiple sizes
- **Automatic Updates**: Sync cart when product prices change
- **Stock Reservation**: Reserve items during checkout process

In [ ]:
# 3.1 Get or Create User Cart
def get_user_cart(user):
    """Get or create cart for user"""
    cart, created = Cart.objects.get_or_create(user=user)
    if created:
        print(f"🆕 New cart created for {user.email}")
    else:
        print(f"📋 Existing cart found for {user.email}")
    return cart

# Get buyer's cart
buyer_cart = get_user_cart(test_data['user'])
print(f"🛒 Cart ID: {buyer_cart.id}")
print(f"📊 Items in cart: {buyer_cart.items.count()}")

# 3.2 Add Product Without Sizes to Cart
def add_product_to_cart(cart, product, quantity, size=None):
    """Add product to cart with stock validation"""
    try:
        # Check stock availability
        if product.has_sizes and size:
            size_obj = product.sizes.get(size=size)
            if size_obj.available_quantity < quantity:
                raise ValidationError(f"Insufficient stock for size {size}")
        elif not product.has_sizes:
            if product.available_quantity < quantity:
                raise ValidationError("Insufficient stock")
        
        # Add or update cart item
        cart_item, created = CartItem.objects.get_or_create(
            cart=cart,
            product=product,
            size=size,
            defaults={
                'quantity': quantity,
                'price_when_added': product.current_price
            }
        )
        
        if not created:
            # Update existing item
            cart_item.quantity += quantity
            cart_item.save()
            print(f"📈 Updated cart item: {product.product_name} (qty: {cart_item.quantity})")
        else:
            print(f"➕ Added to cart: {product.product_name} (qty: {quantity})")
            
        return cart_item
        
    except ValidationError as e:
        print(f"❌ Error adding to cart: {e}")
        return None
    except Exception as e:
        print(f"💥 Unexpected error: {e}")
        return None

# Add laptop to cart (no sizes)
laptop_item = add_product_to_cart(
    cart=buyer_cart,
    product=test_data['product1'],  # Laptop
    quantity=1
)

# Add t-shirt to cart (with size)
tshirt_item = add_product_to_cart(
    cart=buyer_cart,
    product=test_data['product2'],  # T-shirt
    quantity=2,
    size='M'
)

# 3.3 View Cart Contents
def display_cart_contents(cart):
    """Display detailed cart contents"""
    items = cart.items.all()
    total_value = Decimal('0.00')
    
    print(f"\n🛒 Cart Contents for {cart.user.email}")
    print("=" * 50)
    
    for item in items:
        item_total = item.quantity * item.price_when_added
        total_value += item_total
        
        size_info = f" (Size: {item.size})" if item.size else ""
        print(f"📦 {item.product.product_name}{size_info}")
        print(f"   💰 Price: ${item.price_when_added}")
        print(f"   🔢 Quantity: {item.quantity}")
        print(f"   💵 Subtotal: ${item_total}")
        print(f"   📅 Added: {item.created_at.strftime('%Y-%m-%d %H:%M')}")
        print("-" * 30)
    
    print(f"💰 Total Cart Value: ${total_value}")
    print(f"📊 Total Items: {items.count()}")
    return total_value

cart_total = display_cart_contents(buyer_cart)

# 3.4 Update Cart Item Quantity
def update_cart_item_quantity(cart_item, new_quantity):
    """Update cart item quantity with validation"""
    try:
        if new_quantity <= 0:
            cart_item.delete()
            print(f"🗑️ Removed {cart_item.product.product_name} from cart")
            return None
        
        # Validate stock
        if cart_item.product.has_sizes and cart_item.size:
            size_obj = cart_item.product.sizes.get(size=cart_item.size)
            if size_obj.available_quantity < new_quantity:
                raise ValidationError(f"Only {size_obj.available_quantity} items available")
        elif not cart_item.product.has_sizes:
            if cart_item.product.available_quantity < new_quantity:
                raise ValidationError(f"Only {cart_item.product.available_quantity} items available")
        
        old_quantity = cart_item.quantity
        cart_item.quantity = new_quantity
        cart_item.save()
        
        print(f"✏️ Updated {cart_item.product.product_name}: {old_quantity} → {new_quantity}")
        return cart_item
        
    except ValidationError as e:
        print(f"❌ Update failed: {e}")
        return cart_item

# Update t-shirt quantity
if tshirt_item:
    update_cart_item_quantity(tshirt_item, 3)

# 3.5 Remove Item from Cart
def remove_cart_item(cart_item):
    """Remove item from cart"""
    product_name = cart_item.product.product_name
    cart_item.delete()
    print(f"🗑️ Removed {product_name} from cart")

# Show updated cart
print("\n📊 Updated Cart Contents:")
display_cart_contents(buyer_cart)

# 4. 📋 Order Processing Workflow

Show the complete order lifecycle from cart checkout to order creation, including order status updates and validation.

## Order States:
- `under_paying` → Initial state, payment pending
- `processing` → Payment confirmed, preparing order  
- `shipped` → Order dispatched
- `delivered` → Order completed
- `cancelled` → Order cancelled

In [ ]:
# 4.1 Create Order from Cart
@transaction.atomic
def create_order_from_cart(cart):
    """Create order from cart with product history snapshots"""
    try:
        # Validate cart is not empty
        if not cart.items.exists():
            raise ValidationError("Cart is empty")
        
        # Calculate total
        total_amount = sum(
            item.quantity * item.price_when_added 
            for item in cart.items.all()
        )
        
        # Generate unique order ID
        order_id = f"ORD-{uuid.uuid4().hex[:8].upper()}"
        
        # Create order
        order = Order.objects.create(
            order_id=order_id,
            user_id=cart.user,
            total_amount=total_amount,
            status='under_paying',
            shipping_address="123 Demo Street, Khartoum, Sudan",
            notes="Order created from cart checkout"
        )
        
        # Create order items with product snapshots
        for cart_item in cart.items.all():
            # Create product history snapshot
            product_snapshot = ProductHistory.create_from_product(cart_item.product)
            
            # Create order item with snapshot reference
            order.items.create(
                product_snapshot=product_snapshot,
                quantity=cart_item.quantity,
                price=cart_item.price_when_added,
                size=cart_item.size
            )
            
            # Reserve stock
            if cart_item.product.has_sizes and cart_item.size:
                cart_item.product.reserve_size_stock(cart_item.size, cart_item.quantity)
            else:
                cart_item.product.reserve_stock(cart_item.quantity)
        
        # Clear cart after order creation
        cart.items.all().delete()
        
        print(f"✅ Order created successfully!")
        print(f"📋 Order ID: {order.order_id}")
        print(f"💰 Total Amount: ${order.total_amount}")
        print(f"📊 Status: {order.status}")
        
        return order
        
    except ValidationError as e:
        print(f"❌ Order creation failed: {e}")
        return None
    except Exception as e:
        print(f"💥 Unexpected error: {e}")
        return None

# Create order from our test cart
if buyer_cart.items.exists():
    test_order = create_order_from_cart(buyer_cart)
else:
    print("🛒 Cart is empty, adding items first...")
    # Re-add items for demonstration
    add_product_to_cart(buyer_cart, test_data['product1'], 1)
    add_product_to_cart(buyer_cart, test_data['product2'], 2, 'M')
    test_order = create_order_from_cart(buyer_cart)

# 4.2 Order Details Display
def display_order_details(order):
    """Display comprehensive order information"""
    print(f"\n📋 Order Details: {order.order_id}")
    print("=" * 50)
    print(f"👤 Customer: {order.user_id.get_full_name()}")
    print(f"📧 Email: {order.user_id.email}")
    print(f"📅 Order Date: {order.created_at.strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"📊 Status: {order.status}")
    print(f"💰 Total: ${order.total_amount}")
    print(f"🏠 Shipping: {order.shipping_address}")
    
    print(f"\n📦 Order Items:")
    print("-" * 30)
    
    for item in order.items.all():
        size_info = f" (Size: {item.size})" if item.size else ""
        print(f"• {item.product_snapshot.product_name}{size_info}")
        print(f"  💰 Price: ${item.price}")
        print(f"  🔢 Quantity: {item.quantity}")
        print(f"  💵 Subtotal: ${item.price * item.quantity}")
        print(f"  📸 Snapshot: {item.product_snapshot.snapshot_taken_at}")
    
    print("-" * 30)
    print(f"💰 Order Total: ${order.total_amount}")

if test_order:
    display_order_details(test_order)

# 4.3 Update Order Status
def update_order_status(order, new_status, notes=None):
    """Update order status with validation"""
    valid_statuses = ['under_paying', 'processing', 'shipped', 'delivered', 'cancelled']
    
    if new_status not in valid_statuses:
        print(f"❌ Invalid status: {new_status}")
        return False
    
    old_status = order.status
    order.status = new_status
    order.updated_at = timezone.now()
    
    if notes:
        order.notes = f"{order.notes}\n{timezone.now()}: {notes}" if order.notes else notes
    
    order.save()
    
    print(f"✅ Order status updated: {old_status} → {new_status}")
    if notes:
        print(f"📝 Notes: {notes}")
    
    return True

# 4.4 Order Status Workflow Simulation
print("\n🔄 Order Status Workflow Simulation:")
print("=" * 40)

if test_order:
    # Simulate payment confirmation
    update_order_status(test_order, 'processing', 'Payment confirmed via test gateway')
    
    # Simulate shipping
    update_order_status(test_order, 'shipped', 'Order dispatched via courier')
    
    # Simulate delivery
    update_order_status(test_order, 'delivered', 'Order delivered successfully')

# 4.5 Order Search and Filtering
def get_user_orders(user, status=None, limit=10):
    """Get user orders with optional status filtering"""
    orders = Order.objects.filter(user_id=user)
    
    if status:
        orders = orders.filter(status=status)
    
    orders = orders.order_by('-created_at')[:limit]
    
    print(f"\n📋 Orders for {user.email}:")
    print("=" * 40)
    
    for order in orders:
        print(f"🆔 {order.order_id} | 📊 {order.status} | 💰 ${order.total_amount}")
        print(f"   📅 {order.created_at.strftime('%Y-%m-%d %H:%M')}")
    
    return orders

# Display user's orders
user_orders = get_user_orders(test_data['user'])

print(f"\n✅ Order processing workflow completed!")
print(f"📊 Total orders created: {Order.objects.filter(user_id=test_data['user']).count()}")

# 5. 💳 Payment Integration Examples

Implement payment creation, confirmation, and webhook handling with multiple gateway support (test, Stripe, PayPal).

## Payment Gateways Supported:
- **Test Gateway**: For development and testing
- **Stripe**: Credit/debit card processing  
- **PayPal**: Digital wallet payments
- **Cash on Delivery**: Manual payment method

In [ ]:
# 5.1 Create Payment for Order
def create_payment_for_order(order, gateway_name='test_gateway', payment_method='credit_card'):
    """Create payment instance for an order"""
    try:
        # Get payment gateway
        gateway = PaymentGateway.objects.get(name=gateway_name, is_active=True)
        
        # Calculate fees
        base_amount = order.total_amount
        fixed_fee = gateway.fixed_fee or Decimal('0.00')
        percentage_fee = gateway.percentage_fee or Decimal('0.00')
        
        gateway_fee = fixed_fee + (base_amount * percentage_fee / 100)
        total_amount = base_amount + gateway_fee
        
        # Generate payment ID
        payment_id = str(uuid.uuid4())
        
        # Create payment
        payment = Payment.objects.create(
            payment_id=payment_id,
            order_reference=order.order_id,
            user=order.user_id,
            gateway=gateway,
            amount=base_amount,
            gateway_fee=gateway_fee,
            total_amount=total_amount,
            payment_method=payment_method,
            status='pending'
        )
        
        print(f"💳 Payment created successfully!")
        print(f"🆔 Payment ID: {payment.payment_id}")
        print(f"💰 Amount: ${payment.amount}")
        print(f"💸 Gateway Fee: ${payment.gateway_fee}")
        print(f"💵 Total: ${payment.total_amount}")
        print(f"🏦 Gateway: {gateway.name}")
        
        return payment
        
    except PaymentGateway.DoesNotExist:
        print(f"❌ Payment gateway '{gateway_name}' not found")
        return None
    except Exception as e:
        print(f"💥 Payment creation error: {e}")
        return None

# Create payment for our test order
if test_order:
    test_payment = create_payment_for_order(test_order)

# 5.2 Process Payment (Test Gateway)
def process_test_payment(payment, test_card='4242424242424242'):
    """Process payment using test gateway"""
    try:
        # Simulate different test scenarios
        test_scenarios = {
            '4242424242424242': 'success',
            '4000000000000002': 'decline',
            '4000000000009995': 'insufficient_funds',
            '4000000000000069': 'expired_card',
            '4000000000000127': 'incorrect_cvc'
        }
        
        scenario = test_scenarios.get(test_card, 'success')
        
        # Create payment attempt
        attempt = PaymentAttempt.objects.create(
            payment=payment,
            attempt_number=1,
            gateway_response={'test_card': test_card, 'scenario': scenario},
            status='processing'
        )
        
        if scenario == 'success':
            # Successful payment
            payment.status = 'completed'
            payment.gateway_transaction_id = f"test_txn_{uuid.uuid4().hex[:12]}"
            payment.gateway_reference = f"test_ref_{uuid.uuid4().hex[:8]}"
            payment.completed_at = timezone.now()
            payment.save()
            
            attempt.status = 'completed'
            attempt.save()
            
            # Update order status
            order = Order.objects.get(order_id=payment.order_reference)
            order.status = 'processing'
            order.save()
            
            print(f"✅ Payment successful!")
            print(f"🆔 Transaction ID: {payment.gateway_transaction_id}")
            print(f"📋 Order status updated to: {order.status}")
            
        else:
            # Failed payment
            payment.status = 'failed'
            payment.save()
            
            attempt.status = 'failed'
            attempt.failure_reason = f"Test scenario: {scenario}"
            attempt.save()
            
            print(f"❌ Payment failed: {scenario}")
            print(f"🔄 You can retry with a different card")
        
        return payment
        
    except Exception as e:
        print(f"💥 Payment processing error: {e}")
        return None

# Process the test payment
if test_payment:
    processed_payment = process_test_payment(test_payment)

# 5.3 Payment Status Tracking
def get_payment_status(payment_id):
    """Get detailed payment status"""
    try:
        payment = Payment.objects.get(payment_id=payment_id)
        
        print(f"\n💳 Payment Status Report")
        print("=" * 40)
        print(f"🆔 Payment ID: {payment.payment_id}")
        print(f"📋 Order: {payment.order_reference}")
        print(f"💰 Amount: ${payment.amount}")
        print(f"📊 Status: {payment.status}")
        print(f"🏦 Gateway: {payment.gateway.name}")
        print(f"💳 Method: {payment.payment_method}")
        print(f"📅 Created: {payment.created_at.strftime('%Y-%m-%d %H:%M:%S')}")
        
        if payment.completed_at:
            print(f"✅ Completed: {payment.completed_at.strftime('%Y-%m-%d %H:%M:%S')}")
        
        if payment.gateway_transaction_id:
            print(f"🔗 Transaction ID: {payment.gateway_transaction_id}")
        
        # Show payment attempts
        attempts = payment.attempts.all()
        print(f"\n🔄 Payment Attempts ({attempts.count()}):")
        for attempt in attempts:
            print(f"  #{attempt.attempt_number} - {attempt.status}")
            if attempt.failure_reason:
                print(f"    ❌ Reason: {attempt.failure_reason}")
        
        return payment
        
    except Payment.DoesNotExist:
        print(f"❌ Payment {payment_id} not found")
        return None

# Check payment status
if test_payment:
    get_payment_status(test_payment.payment_id)

# 5.4 Payment Refund Processing
def create_refund(payment, amount=None, reason="Customer request"):
    """Create refund for a payment"""
    try:
        if payment.status != 'completed':
            raise ValidationError("Can only refund completed payments")
        
        # Default to full refund
        refund_amount = amount or payment.amount
        
        if refund_amount > payment.amount:
            raise ValidationError("Refund amount cannot exceed payment amount")
        
        # Check if already refunded
        existing_refunds = payment.refunds.aggregate(
            total=sum(refund.amount for refund in payment.refunds.all())
        )['total'] or Decimal('0.00')
        
        if existing_refunds + refund_amount > payment.amount:
            raise ValidationError("Total refunds cannot exceed payment amount")
        
        # Create refund
        refund = Refund.objects.create(
            payment=payment,
            amount=refund_amount,
            reason=reason,
            status='pending',
            gateway_refund_id=f"refund_{uuid.uuid4().hex[:12]}"
        )
        
        # Process refund (simulate)
        refund.status = 'completed'
        refund.processed_at = timezone.now()
        refund.save()
        
        print(f"✅ Refund processed successfully!")
        print(f"💰 Refund Amount: ${refund.amount}")
        print(f"📝 Reason: {refund.reason}")
        print(f"🆔 Refund ID: {refund.gateway_refund_id}")
        
        return refund
        
    except ValidationError as e:
        print(f"❌ Refund failed: {e}")
        return None
    except Exception as e:
        print(f"💥 Refund error: {e}")
        return None

# 5.5 Webhook Simulation
def simulate_payment_webhook(payment, event_type='payment.succeeded'):
    """Simulate payment gateway webhook"""
    webhook_data = {
        'event': event_type,
        'payment_id': payment.payment_id,
        'transaction_id': payment.gateway_transaction_id,
        'amount': str(payment.amount),
        'status': 'completed' if event_type == 'payment.succeeded' else 'failed',
        'timestamp': timezone.now().isoformat()
    }
    
    print(f"📡 Webhook received: {event_type}")
    print(f"📄 Data: {json.dumps(webhook_data, indent=2)}")
    
    # Process webhook
    if event_type == 'payment.succeeded':
        payment.status = 'completed'
        payment.save()
        print("✅ Payment confirmed via webhook")
    elif event_type == 'payment.failed':
        payment.status = 'failed'
        payment.save()
        print("❌ Payment failure confirmed via webhook")
    
    return webhook_data

print("\n💳 Payment integration examples completed!")
print(f"📊 Total payments: {Payment.objects.count()}")
print(f"✅ Successful payments: {Payment.objects.filter(status='completed').count()}")

# 6. 📦 Stock Management and Reservations

Demonstrate atomic stock operations, concurrent transaction handling, and real-time availability checking.

## Stock Operations:
- **Reserve**: Hold stock during checkout
- **Unreserve**: Release stock if payment fails
- **Confirm**: Finalize stock reduction after payment
- **Check Availability**: Real-time stock validation

In [ ]:
# 6.1 Stock Status Display
def display_stock_status(product):
    """Show detailed stock information for a product"""
    print(f"\n📦 Stock Status: {product.product_name}")
    print("=" * 50)
    
    if product.has_sizes:
        print("👕 Product with sizes:")
        total_available = 0
        total_reserved = 0
        
        for size in product.sizes.all():
            print(f"  Size {size.size}:")
            print(f"    ✅ Available: {size.available_quantity}")
            print(f"    🔒 Reserved: {size.reserved_quantity}")
            print(f"    📊 Total: {size.get_total_stock()}")
            total_available += size.available_quantity
            total_reserved += size.reserved_quantity
        
        print(f"\n📊 Total across all sizes:")
        print(f"  ✅ Available: {total_available}")
        print(f"  🔒 Reserved: {total_reserved}")
        print(f"  📈 Grand Total: {total_available + total_reserved}")
    else:
        print("📱 Product without sizes:")
        print(f"  ✅ Available: {product.available_quantity}")
        print(f"  🔒 Reserved: {product.reserved_quantity}")
        print(f"  📊 Total: {product.get_total_stock()}")

# Display initial stock status
display_stock_status(test_data['product1'])  # Laptop
display_stock_status(test_data['product2'])  # T-shirt

# 6.2 Stock Reservation Operations
@transaction.atomic
def reserve_product_stock(product, quantity, size=None):
    """Reserve stock with atomic transaction"""
    try:
        if product.has_sizes and size:
            # Reserve size-specific stock
            product.reserve_size_stock(size, quantity)
            print(f"🔒 Reserved {quantity} units of {product.product_name} (Size: {size})")
        elif not product.has_sizes:
            # Reserve product stock
            product.reserve_stock(quantity)
            print(f"🔒 Reserved {quantity} units of {product.product_name}")
        else:
            raise ValidationError("Size must be specified for products with sizes")
        
        return True
        
    except ValidationError as e:
        print(f"❌ Reservation failed: {e}")
        return False

# Test stock reservation
print("\n🔒 Testing Stock Reservations:")
reserve_product_stock(test_data['product1'], 2)  # Reserve 2 laptops
reserve_product_stock(test_data['product2'], 1, 'L')  # Reserve 1 L t-shirt

# Show updated stock status
display_stock_status(test_data['product1'])
display_stock_status(test_data['product2'])

# 6.3 Stock Unreservation (Payment Timeout)
@transaction.atomic
def unreserve_product_stock(product, quantity, size=None):
    """Unreserve stock when payment fails or times out"""
    try:
        if product.has_sizes and size:
            product.unreserve_size_stock(size, quantity)
            print(f"🔓 Unreserved {quantity} units of {product.product_name} (Size: {size})")
        elif not product.has_sizes:
            product.unreserve_stock(quantity)
            print(f"🔓 Unreserved {quantity} units of {product.product_name}")
        
        return True
        
    except ValidationError as e:
        print(f"❌ Unreservation failed: {e}")
        return False

# 6.4 Confirm Stock Sale (Payment Success)
@transaction.atomic
def confirm_product_sale(product, quantity, size=None):
    """Confirm stock sale after successful payment"""
    try:
        if product.has_sizes and size:
            product.confirm_size_sale(size, quantity)
            print(f"✅ Confirmed sale of {quantity} units of {product.product_name} (Size: {size})")
        elif not product.has_sizes:
            product.confirm_sale(quantity)
            print(f"✅ Confirmed sale of {quantity} units of {product.product_name}")
        
        return True
        
    except ValidationError as e:
        print(f"❌ Sale confirmation failed: {e}")
        return False

# Test sale confirmation
print("\n✅ Testing Sale Confirmations:")
confirm_product_sale(test_data['product1'], 1)  # Confirm 1 laptop sale
confirm_product_sale(test_data['product2'], 1, 'L')  # Confirm 1 L t-shirt sale

# Final stock status
print("\n📊 Final Stock Status:")
display_stock_status(test_data['product1'])
display_stock_status(test_data['product2'])

# 6.5 Concurrent Stock Operations Simulation
def simulate_concurrent_operations():
    """Simulate concurrent stock operations"""
    print("\n⚡ Simulating Concurrent Operations:")
    print("=" * 40)
    
    # Create multiple products for testing
    laptop = test_data['product1']
    
    print(f"Initial laptop stock: {laptop.available_quantity} available, {laptop.reserved_quantity} reserved")
    
    # Simulate multiple users trying to reserve stock simultaneously
    operations = [
        ("User A", 2),
        ("User B", 3), 
        ("User C", 1),
        ("User D", 4)
    ]
    
    successful_reservations = []
    
    for user, qty in operations:
        try:
            with transaction.atomic():
                if laptop.available_quantity >= qty:
                    laptop.reserve_stock(qty)
                    successful_reservations.append((user, qty))
                    print(f"✅ {user}: Reserved {qty} units")
                else:
                    print(f"❌ {user}: Failed to reserve {qty} units (insufficient stock)")
        except Exception as e:
            print(f"💥 {user}: Error - {e}")
    
    print(f"\nFinal state: {laptop.available_quantity} available, {laptop.reserved_quantity} reserved")
    print(f"Successful reservations: {len(successful_reservations)}")
    
    return successful_reservations

# Run concurrent operations simulation
concurrent_results = simulate_concurrent_operations()

print("\n📦 Stock management operations completed!")
print(f"🔒 Total reserved items across all products: {sum(p.get_reserved_stock() for p in Product.objects.all())}")

# 7. 🔄 Real-time Cart Synchronization

Show how cart synchronization works across sessions and automatic product change detection.

# 8. ⏱️ Payment Timer System Implementation  

Implement 15-minute payment windows, automatic stock unreservation on timeout, and payment attempt tracking.

# 9. 📜 Order History and Product Snapshots

Demonstrate product history tracking, price change recording, and order consistency validation.

In [ ]:
# 7.1 Cart Synchronization on Product Changes
def sync_cart_with_product_changes(cart):
    """Synchronize cart items when products change"""
    updated_items = []
    removed_items = []
    
    print(f"\n🔄 Synchronizing cart for {cart.user.email}")
    print("=" * 40)
    
    for cart_item in cart.items.all():
        product = cart_item.product
        
        # Check if product is still available
        if product.is_deleted:
            cart_item.delete()
            removed_items.append(cart_item.product.product_name)
            print(f"🗑️ Removed deleted product: {product.product_name}")
            continue
        
        # Check price changes
        current_price = product.current_price
        if cart_item.price_when_added != current_price:
            old_price = cart_item.price_when_added
            cart_item.price_when_added = current_price
            cart_item.save()
            updated_items.append({
                'product': product.product_name,
                'old_price': old_price,
                'new_price': current_price
            })
            print(f"💰 Updated price for {product.product_name}: ${old_price} → ${current_price}")
        
        # Check stock availability
        if product.has_sizes and cart_item.size:
            try:
                size_obj = product.sizes.get(size=cart_item.size)
                if size_obj.available_quantity < cart_item.quantity:
                    old_qty = cart_item.quantity
                    cart_item.quantity = size_obj.available_quantity
                    cart_item.save()
                    print(f"📉 Reduced quantity for {product.product_name} (Size {cart_item.size}): {old_qty} → {cart_item.quantity}")
            except Size.DoesNotExist:
                cart_item.delete()
                removed_items.append(f"{product.product_name} (Size {cart_item.size})")
                print(f"🗑️ Removed unavailable size: {product.product_name} Size {cart_item.size}")
        elif not product.has_sizes:
            if product.available_quantity < cart_item.quantity:
                old_qty = cart_item.quantity
                cart_item.quantity = product.available_quantity
                cart_item.save()
                print(f"📉 Reduced quantity for {product.product_name}: {old_qty} → {cart_item.quantity}")
    
    return {'updated': updated_items, 'removed': removed_items}

# Test cart synchronization
sync_result = sync_cart_with_product_changes(buyer_cart)

# 8.1 Payment Timer Implementation
class PaymentTimer:
    """Handle payment timeout operations"""
    
    PAYMENT_TIMEOUT_MINUTES = 15
    
    @classmethod
    def start_payment_timer(cls, payment):
        """Start payment timer for order"""
        timeout_time = timezone.now() + timedelta(minutes=cls.PAYMENT_TIMEOUT_MINUTES)
        
        # Store timeout in payment metadata
        payment.metadata = payment.metadata or {}
        payment.metadata['timeout_at'] = timeout_time.isoformat()
        payment.save()
        
        print(f"⏰ Payment timer started for {payment.payment_id}")
        print(f"🕐 Will timeout at: {timeout_time.strftime('%Y-%m-%d %H:%M:%S')}")
        
        return timeout_time
    
    @classmethod
    def check_payment_timeout(cls, payment):
        """Check if payment has timed out"""
        if not payment.metadata or 'timeout_at' not in payment.metadata:
            return False
        
        timeout_time = datetime.fromisoformat(payment.metadata['timeout_at'].replace('Z', '+00:00'))
        is_expired = timezone.now() > timeout_time
        
        if is_expired and payment.status == 'pending':
            cls.handle_payment_timeout(payment)
        
        return is_expired
    
    @classmethod
    def handle_payment_timeout(cls, payment):
        """Handle payment timeout - unreserve stock and update status"""
        try:
            with transaction.atomic():
                # Update payment status
                payment.status = 'expired'
                payment.save()
                
                # Get associated order
                order = Order.objects.get(order_id=payment.order_reference)
                
                # Unreserve stock for all order items
                for order_item in order.items.all():
                    product = Product.objects.get(id=order_item.product_snapshot.product.id)
                    
                    if order_item.size:
                        product.unreserve_size_stock(order_item.size, order_item.quantity)
                    else:
                        product.unreserve_stock(order_item.quantity)
                
                # Update order status
                order.status = 'cancelled'
                order.notes = f"{order.notes}\nPayment timed out at {timezone.now()}"
                order.save()
                
                print(f"⏰ Payment {payment.payment_id} timed out")
                print(f"🔓 Stock unreserved for order {order.order_id}")
                print(f"❌ Order cancelled due to payment timeout")
                
        except Exception as e:
            print(f"💥 Error handling payment timeout: {e}")

# Test payment timer
if test_payment:
    PaymentTimer.start_payment_timer(test_payment)
    
    # Simulate checking timeout (without actual delay)
    print("\n🔍 Checking payment timeout status:")
    is_expired = PaymentTimer.check_payment_timeout(test_payment)
    print(f"Expired: {is_expired}")

# 9.1 Product History Tracking
def demonstrate_product_history_tracking():
    """Show how product history is tracked"""
    print("\n📜 Product History Tracking Demo")
    print("=" * 40)
    
    laptop = test_data['product1']
    
    # Show current product state
    print(f"Current product: {laptop.product_name}")
    print(f"Current price: ${laptop.price}")
    print(f"Available quantity: {laptop.available_quantity}")
    
    # Create initial history snapshot
    initial_history = ProductHistory.create_from_product(laptop)
    print(f"📸 Initial snapshot created: {initial_history.snapshot_taken_at}")
    
    # Simulate product changes
    print("\n🔄 Simulating product changes...")
    
    # Change 1: Update price
    old_price = laptop.price
    laptop.price = Decimal('1399.99')
    laptop.save()
    
    price_history = ProductHistory.create_from_product(laptop)
    print(f"💰 Price changed: ${old_price} → ${laptop.price}")
    print(f"📸 Price change snapshot: {price_history.snapshot_taken_at}")
    
    # Change 2: Update description
    laptop.product_description = "High-performance laptop for coding and gaming"
    laptop.save()
    
    desc_history = ProductHistory.create_from_product(laptop)
    print(f"📝 Description updated")
    print(f"📸 Description change snapshot: {desc_history.snapshot_taken_at}")
    
    # Show all history entries
    print(f"\n📊 Total history entries: {laptop.history.count()}")
    
    for i, history in enumerate(laptop.history.order_by('snapshot_taken_at'), 1):
        print(f"{i}. {history.snapshot_taken_at.strftime('%Y-%m-%d %H:%M:%S')} - Price: ${history.product_price}")
    
    return laptop.history.all()

# Run history tracking demo
history_entries = demonstrate_product_history_tracking()

# 10. Error Handling and Validation
print("\n\n# 10. ⚠️ Error Handling and Validation")
print("\nShow proper error handling for cart operations, payment failures, and stock validation scenarios.")

def demonstrate_error_scenarios():
    """Demonstrate various error scenarios and handling"""
    print("\n⚠️ Error Handling Scenarios")
    print("=" * 40)
    
    laptop = test_data['product1']
    
    # Scenario 1: Insufficient stock
    print("\n1️⃣ Testing insufficient stock scenario:")
    try:
        laptop.reserve_stock(1000)  # Try to reserve more than available
    except ValidationError as e:
        print(f"✅ Caught expected error: {e}")
    
    # Scenario 2: Invalid payment amount
    print("\n2️⃣ Testing invalid payment scenario:")
    try:
        invalid_payment = Payment.objects.create(
            payment_id=str(uuid.uuid4()),
            order_reference='INVALID-ORDER',
            user=test_data['user'],
            gateway=test_data['gateway'],
            amount=Decimal('-10.00'),  # Negative amount
            total_amount=Decimal('-10.00')
        )
    except Exception as e:
        print(f"✅ Caught payment validation error: {e}")
    
    # Scenario 3: Cart validation
    print("\n3️⃣ Testing cart validation:")
    empty_cart = Cart.objects.create(user=test_data['user'])
    try:
        create_order_from_cart(empty_cart)  # Try to create order from empty cart
    except Exception as e:
        print(f"✅ Caught empty cart error: Cart validation working")
    
    # Scenario 4: Size not found
    print("\n4️⃣ Testing size validation:")
    try:
        test_data['product2'].reserve_size_stock('XXL', 1)  # Invalid size
    except ValidationError as e:
        print(f"✅ Caught size validation error: {e}")
    
    print("\n✅ All error scenarios handled correctly!")

# Run error scenarios
demonstrate_error_scenarios()

# Final Summary
print("\n\n🎉 E-Commerce System Usage Guide Complete!")
print("=" * 50)
print("✅ Cart Management: Add, update, sync operations")
print("✅ Order Processing: Complete lifecycle management") 
print("✅ Payment Integration: Multi-gateway support")
print("✅ Stock Management: Atomic operations and reservations")
print("✅ Real-time Sync: Cart and product change detection")
print("✅ Payment Timers: Timeout handling and stock unreservation")
print("✅ History Tracking: Product snapshots and audit trails")
print("✅ Error Handling: Comprehensive validation and recovery")

print(f"\n📊 Final Statistics:")
print(f"👥 Users: {User.objects.count()}")
print(f"📦 Products: {Product.objects.count()}")
print(f"🛒 Carts: {Cart.objects.count()}")
print(f"📋 Orders: {Order.objects.count()}")
print(f"💳 Payments: {Payment.objects.count()}")
print(f"📜 Product History: {ProductHistory.objects.count()}")

print(f"\n🚀 System is ready for production use!")

# 🌐 Complete API Endpoints Reference

This section provides a comprehensive list of all API endpoints needed for the E-Commerce system covering **Carts**, **Orders**, and **Payments**.

## 📋 Base URL Structure
```
Base URL: /api/v1/
Authentication: Bearer Token (JWT)
Content-Type: application/json
```

---

## 🛒 Cart Management Endpoints

### **Core Cart Operations**

| Method | Endpoint | Description | Auth Required |
|--------|----------|-------------|---------------|
| `GET` | `/carts/` | Get user's cart with all items | ✅ |
| `GET` | `/carts/count/` | Get cart item count | ✅ |
| `POST` | `/carts/add/` | Add product to cart | ✅ |
| `PUT` | `/carts/update/{cart_item_id}/` | Update cart item quantity | ✅ |
| `DELETE` | `/carts/remove/{cart_item_id}/` | Remove specific item from cart | ✅ |
| `DELETE` | `/carts/clear/` | Clear entire cart | ✅ |

### **Cart Validation & Checkout**

| Method | Endpoint | Description | Auth Required |
|--------|----------|-------------|---------------|
| `POST` | `/carts/validate/` | Validate cart items (stock, prices) | ✅ |
| `POST` | `/carts/checkout/single/` | Checkout single item (direct purchase) | ✅ |
| `POST` | `/carts/checkout/full/` | Checkout entire cart | ✅ |

---

## 📋 Order Management Endpoints

### **Order Lifecycle** 

| Method | Endpoint | Description | Auth Required |
|--------|----------|-------------|---------------|
| `GET` | `/orders/` | List user's orders | ✅ |
| `GET` | `/orders/{order_id}/` | Get specific order details | ✅ |
| `POST` | `/orders/create/` | Create order from cart | ✅ |
| `PUT` | `/orders/{order_id}/status/` | Update order status | ✅ (Owner) |
| `POST` | `/orders/{order_id}/cancel/` | Cancel order | ✅ |

### **Payment & Timer Management**

| Method | Endpoint | Description | Auth Required |
|--------|----------|-------------|---------------|
| `GET` | `/orders/payment-status/{order_id}/` | Check payment timer status | ✅ |
| `GET` | `/orders/check-payment/{order_id}/` | Check order payment status | ✅ |

### **Admin Order Management**

| Method | Endpoint | Description | Auth Required |
|--------|----------|-------------|---------------|
| `POST` | `/orders/admin/cleanup/expired/` | Manually cleanup expired orders | 🔑 Admin |
| `GET` | `/orders/admin/expired-count/` | Get count of expired orders | 🔑 Admin |
| `POST` | `/orders/admin/trigger-cleanup-task/` | Trigger cleanup background task | 🔑 Admin |

---

## 💳 Payment Processing Endpoints

### **Payment Creation & Processing**

| Method | Endpoint | Description | Auth Required |
|--------|----------|-------------|---------------|
| `POST` | `/payments/create/` | Create payment for order | ✅ |
| `POST` | `/payments/process/` | Process payment (gateway) | ✅ |
| `GET` | `/payments/status/{order_reference}/` | Get payment status by order | ✅ |
| `GET` | `/payments/detail/{payment_id}/` | Get payment details | ✅ |
| `GET` | `/payments/my-payments/` | List user's payments | ✅ |

### **Payment Gateway Management**

| Method | Endpoint | Description | Auth Required |
|--------|----------|-------------|---------------|
| `GET` | `/payments/gateways/` | List available payment gateways | ✅ |
| `GET` | `/payments/config/` | Get payment configuration | ✅ |

### **Refund Management**

| Method | Endpoint | Description | Auth Required |
|--------|----------|-------------|---------------|
| `POST` | `/payments/refunds/create/` | Create refund for payment | 🔑 Admin |
| `GET` | `/payments/refunds/` | List refunds | 🔑 Admin |
| `GET` | `/payments/refunds/{refund_id}/` | Get refund details | 🔑 Admin |

### **Testing & Development**

| Method | Endpoint | Description | Auth Required |
|--------|----------|-------------|---------------|
| `POST` | `/payments/test/` | Test payment processing | ✅ (Test Mode) |
| `POST` | `/payments/test/webhook/` | Simulate payment webhook | ✅ (Test Mode) |

---

## 📱 Additional Required Endpoints

Based on the e-commerce workflow, these additional endpoints are recommended:

### **Product Integration**

| Method | Endpoint | Description | Auth Required |
|--------|----------|-------------|---------------|
| `GET` | `/products/` | List products with stock info | ❌ |
| `GET` | `/products/{product_id}/` | Get product details | ❌ |
| `GET` | `/products/{product_id}/stock/` | Check product stock availability | ❌ |
| `GET` | `/products/{product_id}/history/` | Get product price history | ✅ |

### **User Account Integration**

| Method | Endpoint | Description | Auth Required |
|--------|----------|-------------|---------------|
| `GET` | `/accounts/profile/` | Get user profile | ✅ |
| `PUT` | `/accounts/addresses/` | Manage shipping addresses | ✅ |
| `GET` | `/accounts/order-history/` | Get user's order history | ✅ |

### **Real-time Updates**

| Method | Endpoint | Description | Auth Required |
|--------|----------|-------------|---------------|
| `WebSocket` | `/ws/cart/{user_id}/` | Real-time cart updates | ✅ |
| `WebSocket` | `/ws/orders/{user_id}/` | Real-time order status | ✅ |
| `WebSocket` | `/ws/payments/{payment_id}/` | Real-time payment status | ✅ |

---

## 🔄 Complete E-Commerce Flow Endpoints

### **1. Shopping Flow**
```
1. GET /products/ → Browse products
2. POST /carts/add/ → Add to cart
3. GET /carts/ → View cart
4. POST /carts/validate/ → Validate before checkout
5. POST /carts/checkout/full/ → Create order
```

### **2. Payment Flow**
```
6. POST /payments/create/ → Create payment
7. POST /payments/process/ → Process payment
8. GET /payments/status/{order_ref}/ → Check status
9. GET /orders/payment-status/{order_id}/ → Check timer
```

### **3. Order Management Flow**
```
10. GET /orders/{order_id}/ → View order details
11. PUT /orders/{order_id}/status/ → Update status (seller)
12. GET /accounts/order-history/ → View all orders
```

---

## ⚙️ Endpoint Implementation Status

| Module | Total Endpoints | ✅ Implemented | ⚠️ Partial | ❌ Missing |
|--------|----------------|-------------|-----------|----------|
| **Carts** | 8 | 8 | 0 | 0 |
| **Orders** | 8 | 5 | 2 | 1 |
| **Payments** | 12 | 10 | 2 | 0 |
| **Integration** | 10 | 3 | 4 | 3 |

### **Priority Implementation List**

#### 🚨 **High Priority (Missing Core Functionality)**
1. `GET /orders/` - List user orders
2. `GET /orders/{order_id}/` - Order details
3. `POST /orders/create/` - Create order from cart
4. `PUT /orders/{order_id}/status/` - Update order status
5. `POST /orders/{order_id}/cancel/` - Cancel order

#### ⚠️ **Medium Priority (Enhanced Features)**
1. `GET /payments/refunds/` - List refunds
2. `GET /payments/refunds/{refund_id}/` - Refund details
3. `WebSocket` endpoints for real-time updates
4. `GET /products/{product_id}/stock/` - Stock checking

#### 📈 **Low Priority (Nice to Have)**
1. Advanced filtering for order/payment lists
2. Bulk operations for cart management
3. Analytics endpoints for admin dashboard

---

**Next Step**: Implement the missing high-priority endpoints to complete the core e-commerce functionality! 🚀